# Part 4 -- Get Target Data

Get stock data for S&P 500 using **pandas**.

### Load lib codes:

In [8]:
!pwd

/home/jovyan/work/Portfolio/predicting_stock_market_trends_with_Twitter/ipynb


In [9]:
from os import chdir
chdir('/home/jovyan/work/Portfolio/predicting_stock_market_trends_with_Twitter/')

from lib import *
# suppress_warnings()
from lib.twitter_keys import my_keys

In [882]:
!pip install pandas-datareader

You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [862]:
import datetime as dt
import pandas_datareader.data as web
import bs4 as bs
import pickle
import requests

### Web-scrape most recent list of S&P 500 stocks from Wikipedia:

In [863]:
# def get_data_from_google():
#     with open('sp500tickers.pickle', 'rb') as f:
#         tickers = pickle.load(f)
# if not os.path.exists('stocks_df'):
#     os.makedirs('stocks_df')

# start = dt.datetime(2010,6,1)
# end = dt.datetime(2017,6,1)

# for ticker in tickers:
#     print(ticker)
#     if not os.path.exists('stocks_df/{}.csv'.format(ticker)):
#         df = web.DataReader(ticker, 'google', start, end)
#         df.to_csv('stocks_df/{}.csv'.format(ticker))
#     else:
#         print('Already have {}'.format(ticker))

In [864]:
# def save_sp500_tickers():
#     resp = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
#     soup = bs.BeautifulSoup(resp.text, 'html5lib')
#     table = soup.find('table', {'class': 'wikitable sortable'})
#     tickers = []
#     for row in table.findAll('tr')[1:]:
#         ticker = row.findAll('td')[0].text
#         tickers.append(ticker)
#     with open('sp500tickers.pickle', 'wb') as f:
#         pickle.dump(tickers, f)
#     return tickers

In [1]:
!pip install pymongo
import pymongo

You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
#identify port for security
cli = pymongo.MongoClient(host='34.211.171.54', port=27016)

In [5]:
# Instantiates when you put data in 
sp500_collection = cli.stocks_db.sp500_collection
stocks_collection = cli.stocks_db.stocks_collection
cli.stocks_db.collection_names()

['sp500_collection', 'stocks_collection']

In [6]:
print(sp500_collection.count())
print(stocks_collection.count())

505
250633


In [849]:
# sp500_collection.drop()
# stocks_collection.drop()

In [850]:
# def get_sp500_to_mongo():
#     resp = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
#     soup = bs.BeautifulSoup(resp.text, 'html5lib')
#     table = soup.find('table', {'class': 'wikitable sortable'})
#     for row in table.findAll('tr')[1:]:
#         stock = row.findAll('td')[0].text
#         sp500_dict = {
#             'stock': stock,
#             'status': 'pending'
#         }
#         sp500_collection.insert_one(sp500_dict)
# get_sp500_to_mongo()

In [10]:
# Getting ALL data out of a collection
sp500_list = [document for document in sp500_collection.find()]
sp500_df = pd.DataFrame(sp500_list)
sp500_df[sp500_df['status']=='failed']

,_id,status,stock
33,593751512c7311003e9e86a0,failed,AMT
285,593751902c7311003e9f6f08,failed,LMT
329,5937519ed8b1410090fe896b,failed,NWL
338,593751a12c7311003e9f9c96,failed,NBL


In [11]:
# Getting ALL data out of a collection
sp500_list = [document for document in sp500_collection.find()]
sp500_df = pd.DataFrame(sp500_list)
sp500_df[sp500_df['stock']=='AMT']

,_id,status,stock
33,593751512c7311003e9e86a0,failed,AMT


In [871]:
#takes argument of a stock and returns dataframe with stock column:
def get_stocks(stock):
    try:
        start = dt.datetime(2015,6,1)
        end = dt.datetime(2017,6,1)
        df = web.DataReader(stock, 'google', start, end)
        df = df.reset_index()
        df['Stock'] = stock
        print('{} added for {}'.format(len(df), stock))
        return df
    except Exception as e:
        print('Error: e')
        pass

#takes argument of a dataframe AND a mongo collection, turns it into a list of dictionaries and insert into collection:
def df_to_mongo(df, collection):
    try:
        stock_dict_list = list(df.to_dict(orient='index').values())
        collection.insert_many(stock_dict_list)
        sp500_collection.insert_one({'stock': stock, 'status': 'complete'})
        print('Successfully added to mongo')
    except Exception as e:
        sp500_collection.insert_one({'stock': stock, 'status': 'failed'})
        print('Error: e')
        pass

#third function that does everything using both functions:
def get_stocks_to_mongo(stock):
    df = get_stocks(stock)
    df_to_mongo(df, stocks_collection)

In [875]:
# from datetime import datetime
# start = datetime.now()

# while True:
#     # Get a user from the mongo collection
#     sp500 = sp500_collection.find_one_and_delete({'status': 'pending'})

#     if type(sp500) != dict:
#         print('Done!')
#         break

#     # Assign the stock to stock
#     stock = sp500['stock']
        
#     get_stocks_to_mongo(stock)

# end = datetime.now()
# print(end - start)

506 added for MMM
Successfully added to mongo
506 added for ABT
Successfully added to mongo
506 added for ABBV
Successfully added to mongo
506 added for ACN
Successfully added to mongo
506 added for ATVI
Successfully added to mongo
506 added for AYI
Successfully added to mongo
506 added for ADBE
Successfully added to mongo
506 added for AMD
Successfully added to mongo
506 added for AAP
Successfully added to mongo
506 added for AES
Successfully added to mongo
506 added for AET
Successfully added to mongo
506 added for AMG
Successfully added to mongo
506 added for AFL
Successfully added to mongo
506 added for A
Successfully added to mongo
506 added for APD
Successfully added to mongo
506 added for AKAM
Successfully added to mongo
506 added for ALB
Successfully added to mongo
506 added for ARE
Successfully added to mongo
506 added for AGN
Successfully added to mongo
506 added for LNT
Successfully added to mongo
506 added for ALL
Successfully added to mongo
506 added for GOOG
Successfully 

506 added for NTAP
Successfully added to mongo
Error: e
Error: e
506 added for NEM
Successfully added to mongo
506 added for NWSA
Successfully added to mongo
506 added for NWS
Successfully added to mongo
442 added for NLSN
Successfully added to mongo
506 added for NI
Successfully added to mongo
506 added for JWN
Successfully added to mongo
505 added for NTRS
Successfully added to mongo
506 added for NRG
Successfully added to mongo
506 added for NVDA
Successfully added to mongo
506 added for OXY
Successfully added to mongo
506 added for OKE
Successfully added to mongo
506 added for ORCL
Successfully added to mongo
506 added for PCAR
Successfully added to mongo
506 added for PH
Successfully added to mongo
506 added for PAYX
Successfully added to mongo
506 added for PNR
Successfully added to mongo
506 added for PKI
Successfully added to mongo
506 added for PRGO
Successfully added to mongo
506 added for PCG
Successfully added to mongo
506 added for PM
Successfully added to mongo
506 added 

In [12]:
print(sp500_collection.count())
print(stocks_collection.count())

505
250633


In [877]:
250633.0/253000.0
# Got 99% of the data we expected

0.9906442687747036

In [16]:
# Getting ALL data out of a collection
stocks_list = [document for document in stocks_collection.find()]
stocks_df = pd.DataFrame(stocks_list)
stocks_df.head()

,Close,Date,High,Low,Open,Stock,Volume,_id
0,158.98,2015-06-01,159.76,158.57,159.10,MMM,1906559,59375144d8b1410090fd2a88
1,159.08,2015-06-02,159.88,157.95,158.28,MMM,2481192,59375144d8b1410090fd2a89
2,160.16,2015-06-03,160.50,158.98,159.61,MMM,2074495,59375144d8b1410090fd2a8a
3,158.35,2015-06-04,160.38,158.09,159.51,MMM,2237628,59375144d8b1410090fd2a8b
4,157.08,2015-06-05,157.97,156.97,157.50,MMM,2314771,59375144d8b1410090fd2a8c


In [18]:
stocks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250633 entries, 0 to 250632
Data columns (total 8 columns):
Close     250633 non-null float64
Date      250633 non-null datetime64[ns]
High      250630 non-null float64
Low       250630 non-null float64
Open      250630 non-null float64
Stock     250633 non-null object
Volume    250633 non-null int64
_id       250633 non-null object
dtypes: datetime64[ns](1), float64(4), int64(1), object(2)
memory usage: 15.3+ MB
